In [16]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import datasets, ensemble
from sklearn.metrics import mean_squared_error

In [17]:
data=pd.read_csv('datacleaned.csv')
data=data.drop(['Unnamed: 0'], axis=1)

# Only taking data from gameweeks 5+

In [18]:
playerlist=list(data['playerid'].unique())

In [20]:
pointsdict={}
fdrdict={}
for i in playerlist:
    ptslist=[]
    fdrlist=[]
    selection=data[data['playerid']==i]
    for gw in range(1,39):
        ptslist.append(selection[selection['GW']==gw].iloc[0]['total_points'])
        fdrlist.append(selection[selection['GW']==gw].iloc[0]['fdr'])
    pointsdict[i]=ptslist
    fdrdict[i]=fdrlist

In [27]:
pointsdict[618][23]

0

In [6]:
data.insert(0, 'form', 0)

In [7]:
data.insert(0, 'prevscore', 0)
data.insert(0, 'prevfdr', 0)
data.insert(0, 'prev2score', 0)
data.insert(0, 'prev2fdr', 0)
data.insert(0, 'prev3score', 0)
data.insert(0, 'prev3fdr', 0)

In [8]:
def calcForm(scores:list)->int:
    weights=[1,2,3,4,5]
    return sum([x*y for x,y in zip(scores,weights)])

In [30]:
prevscorelist = []
prev2scorelist = []
prev3scorelist = []
prevfdrlist = []
prev2fdrlist = []
prev3fdrlist = []

for index, row in data.iterrows():
    currgw=row['GW']
    
    prevscorelist.append(pointsdict[row['playerid']][currgw-2])
    prev2scorelist.append(pointsdict[row['playerid']][currgw-3])
    prev3scorelist.append(pointsdict[row['playerid']][currgw-4])
    prevfdrlist.append(fdrdict[row['playerid']][currgw-2])
    prev2fdrlist.append(fdrdict[row['playerid']][currgw-3])
    prev3fdrlist.append(fdrdict[row['playerid']][currgw-4])

In [31]:
data['prevscore']=prevscorelist
data['prev2score']=prev2scorelist
data['prev3score']=prev3scorelist
data['prevfdr']=prevfdrlist
data['prev2fdr']=prev2fdrlist
data['prev3fdr']=prev3fdrlist

In [34]:
data.columns

Index(['team', 'playerid', 'name', 'assists', 'clean_sheets', 'creativity',
       'goals_conceded', 'goals_scored', 'ict_index', 'influence', 'minutes',
       'opponent_team', 'round', 'saves', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'value', 'was_home', 'yellow_cards', 'GW',
       'fdr', 'prevscore', 'prev2score', 'prev3score', 'prevfdr', 'prev2fdr',
       'prev3fdr'],
      dtype='object')

In [35]:
feature_cols=['team', 'playerid', 'name', 'minutes', 'opponent_team', 'round', 'value', 
              'was_home', 'yellow_cards', 'GW', 'fdr', 'prevscore', 'prev2score', 
              'prev3score', 'prevfdr', 'prev2fdr', 'prev3fdr']